In [111]:
#import twt_cred as mykey # my credentials twitter
import tweepy
#import csv
#import pandas as pd
####input your credentials here
consumer_key = ''
consumer_secret =''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


In [203]:
import sys
import jsonpickle
import os

searchQuery = '@jimmyfallon' #'#someHashtag'  # this is what we're searching for
maxTweets = 10000# Some arbitrary large number
tweetsPerQry = 100 # this is the max the API permits
fName = 'tweets.txt' # We'll store the tweets in a text file.


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1L

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.user_timeline(screen_name = searchQuery, # user_id=tweeters,
                                include_rts=True,
                                exclude_replies =False,
                                count=tweetsPerQry)
                else:
                    new_tweets = api.user_timeline(screen_name = searchQuery, # user_id=tweeters,
                                include_rts=True,
                                exclude_replies =False,
                                count=tweetsPerQry,
                                max_id=str(max_id - 1),
                                since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.user_timeline(screen_name = searchQuery, # user_id=tweeters,
                                include_rts=True,
                                exclude_replies =False, 
                                count=tweetsPerQry,
                                max_id=str(max_id - 1))
                else:
                    new_tweets = api.user_timeline(screen_name = searchQuery, # user_id=tweeters,
                                include_rts=True,
                                exclude_replies =False, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                        '\n')
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1100 tweets
Downloaded 1200 tweets
Downloaded 1300 tweets
Downloaded 1400 tweets
Downloaded 1500 tweets
Downloaded 1600 tweets
Downloaded 1700 tweets
Downloaded 1799 tweets
Downloaded 1899 tweets
Downloaded 1999 tweets
Downloaded 2099 tweets
Downloaded 2199 tweets
Downloaded 2299 tweets
Downloaded 2399 tweets
Downloaded 2499 tweets
Downloaded 2599 tweets
Downloaded 2699 tweets
Downloaded 2799 tweets
Downloaded 2899 tweets
Downloaded 2999 tweets
Downloaded 3099 tweets
Downloaded 3199 tweets
Downloaded 3220 tweets
No more tweets found
Downloaded 3220 tweets, Saved to tweets.txt


In [204]:
import json
import pandas as pd
import matplotlib.pyplot as plt


In [205]:
tweets_data_path = 'tweets.txt'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [206]:
print len(tweets_data)

3220


In [286]:
tweets_data[1]

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Sun Jul 08 15:07:28 +0000 2018',
 u'entities': {u'hashtags': [{u'indices': [75, 98],
    u'text': u'TonightShowSummerReads'}],
  u'symbols': [],
  u'urls': [{u'display_url': u'soundcloud.com/tonightshowsum\u2026',
    u'expanded_url': u'https://soundcloud.com/tonightshowsummerreads/chapters3-5',
    u'indices': [51, 74],
    u'url': u'https://t.co/HBIGe1A2iz'}],
  u'user_mentions': []},
 u'favorite_count': 1034,
 u'favorited': False,
 u'geo': None,
 u'id': 1015975660890796032L,
 u'id_str': u'1015975660890796032',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_status_id_str': None,
 u'in_reply_to_user_id': None,
 u'in_reply_to_user_id_str': None,
 u'is_quote_status': False,
 u'lang': u'en',
 u'place': None,
 u'possibly_sensitive': False,
 u'retweet_count': 127,
 u'retweeted': False,
 u'source': u'<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 u'text': u'Chapters 3-

In [262]:
tweets = pd.DataFrame() #df
tweets['TimeStamp'] = map(lambda tweet: tweet['created_at'], tweets_data) #
tweets['Name'] = map(lambda tweet: tweet["user"]["screen_name"], tweets_data) #
tweets['text'] = map(lambda tweet: tweet['text'], tweets_data) # map fuction like apply in R function, data
tweets['lang'] = map(lambda tweet: tweet['lang'], tweets_data)
tweets['tweet_id'] = map(lambda tweet: tweet['id'], tweets_data)
tweets['Favorites'] = map(lambda tweet: tweet['favorite_count'], tweets_data)
tweets['Retweet'] = map(lambda tweet: tweet['retweet_count'], tweets_data)
tweets['in_reply_to_status_id_str'] =map(lambda tweet: tweet['in_reply_to_status_id_str'], tweets_data)
tweets['in_reply_to_screen_name'] =map(lambda tweet: tweet['in_reply_to_screen_name'], tweets_data)

In [263]:
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from matplotlib import pyplot as plt

In [264]:
sentiment_analyzer = SentimentIntensityAnalyzer()
def tweet_scale(score):
    if score <= -.6:
        return "most_negative"
    if score <= -.2:
        return "negative"
    if score <= .2:
        return "neutral"
    if score <= .6:
        return "positive"
    if score <= .8:
        return "more_positive"
    if score <= 1:
        return "most_positive"

In [265]:
def polarity_tweets(tweets):
    sentiment = []
    score = []
    for tweet in tweets:
        ss = sentiment_analyzer.polarity_scores(str(tweet))
        sentiment.append(tweet_scale(ss['compound']))
        score.append(ss['compound'])
    
    return sentiment, score

sentiment, score = polarity_tweets(tweets_data)    

tweets['Sentiment'] = sentiment
tweets['Score'] = score

In [266]:
tweets.head()

,TimeStamp,Name,text,lang,tweet_id,Favorites,Retweet,in_reply_to_status_id_str,in_reply_to_screen_name,Sentiment,Score
0,Sun Jul 08 15:09:06 +0000 2018,jimmyfallon,And share photos of where you are reading and ...,en,1015976073576804352,714,76,1015975660890796032,jimmyfallon,most_positive,0.9183
1,Sun Jul 08 15:07:28 +0000 2018,jimmyfallon,"Chapters 3-5 recap of ""Children of Blood and B...",en,1015975660890796032,1034,127,None,None,most_positive,0.8807
2,Sun Jul 08 14:24:03 +0000 2018,jimmyfallon,Gonna put out a sound cloud soon to check in. ...,en,1015964735039262721,866,58,None,None,most_positive,0.9778
3,Sun Jul 08 14:21:35 +0000 2018,jimmyfallon,Oh I’m in deep now. But I’ll be checking in to...,en,1015964114202583040,1316,91,None,None,most_positive,0.9441
4,Sun Jul 08 14:11:54 +0000 2018,jimmyfallon,Well if that’s not a great review. It’s a fun ...,en,1015961678964494336,1229,109,None,None,most_positive,0.9855


In [278]:
#Sentiment Count of Tweets 
tweets.groupby(['Sentiment'])['Score'].count().reset_index().sort_values(['Score'], ascending = False)

,Sentiment,Score
2,most_positive,3154
0,more_positive,60
4,positive,4
1,most_negative,1
3,neutral,1


In [275]:
#Sentiment By Mean "Favorites"
tweets.groupby(['Sentiment'])['Favorites'].mean().reset_index().sort_values(['Favorites'], ascending = False)


,Sentiment,Favorites
2,most_positive,2887.913126
0,more_positive,2629.216667
4,positive,1879.000000
1,most_negative,647.000000
3,neutral,0.000000


In [279]:
#Sentiment By Mean "Retweets"
tweets.groupby(['Sentiment'])['Retweet'].mean().reset_index().sort_values(['Retweet'], ascending = False)

,Sentiment,Retweet
3,neutral,952.000000
2,most_positive,868.004439
0,more_positive,694.633333
4,positive,269.750000
1,most_negative,69.000000


In [283]:
#most positive/ most negative tweets
sentiment = tweets.sort_values(['Score'], ascending = False).reset_index()
most_positive = sentiment .iloc[0]
most_negative = sentiment .iloc[(len(test)-1)]

In [284]:
most_positive

index                                                                     1255
TimeStamp                                       Fri Apr 07 00:49:31 +0000 2017
Name                                                               jimmyfallon
text                         RT @TheRock: The. Best. Time. Hot as f*ck in t...
lang                                                                        en
tweet_id                                                    850148499614314496
Favorites                                                                    0
Retweet                                                                    492
in_reply_to_status_id_str                                                 None
in_reply_to_screen_name                                                   None
Sentiment                                                        most_positive
Score                                                                   0.9982
Name: 0, dtype: object

In [285]:
most_negative

index                                                                      239
TimeStamp                                       Thu Apr 26 04:29:24 +0000 2018
Name                                                               jimmyfallon
text                         Nono no no no no no!!!!! No no no no no no no ...
lang                                                                        it
tweet_id                                                    989360772739461120
Favorites                                                                  647
Retweet                                                                     69
in_reply_to_status_id_str                                                 None
in_reply_to_screen_name                                                   None
Sentiment                                                        most_negative
Score                                                                  -0.9227
Name: 3219, dtype: object

# Sources

http://adilmoujahid.com/posts/2014/07/twitter-analytics/


https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively./

https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
